In [1]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import pickle
import os
import glob

In [2]:
df = pd.read_hdf("features/mfcc_test/timit.hdf")
features = np.array(df["features"].tolist())
label = np.array(df["labels"].tolist())
df1 = pd.DataFrame(features)
df2 = pd.DataFrame(label)
df = pd.merge(df1, df2, right_index=True, left_index=True)
df.rename(columns={'0_y': 'Label'}, inplace=True)
print(df.head())

        0_x          1          2          3         4          5          6  \
0  7.347483 -31.977038   0.555148  -2.712652  4.714649  -1.503090   5.682993   
1  7.120237 -25.447563   2.613370  -8.789230 -8.813480 -16.433003 -10.110884   
2  7.270163 -25.650864   3.908348 -11.277479 -7.791231 -17.048360 -12.722343   
3  7.656876 -15.387372  -6.042067 -26.001142 -4.685439   2.421713  10.513782   
4  7.596893 -19.993179 -12.438937 -30.808372 -9.961764   1.546564  11.479846   

          7          8          9         10         11        12 Label  
0  7.735081   5.687700  -1.909077  -7.926409  -3.457316 -9.001497   sil  
1  3.511426   7.033892  10.669727  14.093879  12.479116  8.000335   sil  
2 -5.906701   6.482596   9.179485  15.817018   7.228364  9.730352   sil  
3 -1.394585 -12.179339 -11.878265  14.139992  13.451713  5.799781   sil  
4  7.013714  -8.286893  -5.867128  18.251741  21.779664  7.973898   sil  


In [3]:
perCt =0
totalCt =0

def calPER(x,lbl):
    global perCt
    global totalCt
    totalCt += 1
    if x[lbl] != x['Label']:
        #print(x[lbl],x['Label'])
        if(len(x[lbl]) == 0 and len(x['Label'])!= 0 ) or (len(x[lbl]) != 0 and len(x['Label'])== 0 ) or (len(x[lbl]) != 0 and len(x['Label'])!= 0 ):
            perCt += 1

        
test_df = df.iloc[:,:-1]
for i in range(1,8):
    filename = ''.join(['models/',str(2**i),'M_GMM\\','*.pkl'])
    eval_df = pd.DataFrame()
    for file in glob.glob(filename):
        #print(file)
        with open(file, "rb") as f:
            gmm = pickle.load(f)
            score = gmm.score_samples(test_df)
            label = os.path.basename(file).split('_')[0]
            eval_df[label] = score
    eval_label = ''.join(['eval_score_',str(2**i),'M_GMM'])
    df[eval_label] = eval_df.idxmax(axis=1)
    df.apply(calPER,axis=1,args=(eval_label,))
    print('PER for',str(2**i),'M_GMM',perCt/totalCt)
    perCt =0
    totalCt =0


PER for 2 M_GMM 0.8871422552101903
PER for 4 M_GMM 0.88587819660973
PER for 8 M_GMM 0.8778400803811622
PER for 16 M_GMM 0.8833824911677957
PER for 32 M_GMM 0.8752795514212557
PER for 64 M_GMM 0.884452079214339
PER for 128 M_GMM 0.8807895504489028
